# SQL vs NoSQL 
## PostgreSQL vs MongoDB
<br>
<br>
Adam Brzyzek 
<br>
<br>
<br>
Tech Upskill Meeting 20.02.2024

# SQL Database vs NoSQL Database

# Data Storage Model

SQL: Data are stored in relationed tables with fixed rows and columns.


NoSQL:
- Document - using JSON documents to storage data. example: MongoDB, CouchDB
- Key-Value - data stored in key-value pairs. example: Redis, Amazon DynamoDB
- Graph example - data are stored in nodes and edges. example: Neo4j, Amazon Neptune 
- Wide-column - organizes related data in column families rather than traditional rows. example: Cassandra, Hbase


Document: general purpose, 
Key-value: large amounts of data with simple lookup queries, 
Wide-column: large amounts of data with predictable query patterns, 
Graph: analyzing and traversing relationships between connected data

ObjectId BSON jest to typ danych 12-bajtowy, ma następujący format:
Pierwsze cztery bajty datownik
Kolejne trzy bajtów kodu maszynowego jest
Natychmiast dwa bajty złożone przez identyfikator procesu (PID)
Przez ostatnie trzy bajty liczb losowych.

# Schemas

SQL: Ridge

NoSQL: Flexible


# Scaling

SQL: Vertical (scale-up with a larger server)

NoSQL: Horizontal (scale-out across commodity servers)

# Joins

SQL: Typically required

NoSQL: Typically not required

# Multi-Record ACID Transactions

SQL: Supported

NoSQL: Most do not support multi-record ACID transactions.

# ACID

ACID is an acronym that stands for atomicity, consistency, isolation, and durability.

- **Atomicity** - Atomicity ensures that all steps in a single database transaction are either fully-completed or reverted to their original state. 
- **Consistency** - Consistency ensures that every transaction in a database brings it from one valid state to another.
- **Isolation** - Multiple transactions occur simultaneously, isolation ensures that each one is executed in a way that it’s unaware of other concurrent transactions.
- **Durability** - Ensures that changes to your data made by successfully executed transactions will be saved, even in the event of system failure.

# BASE

BASE is an acronym basically available, soft state, and eventually consistent.

- **Basically available** - The system is always available, however, this does not guarantee full data availability in every case.
- **Soft state** - Data in the system may be temporarily inconsistent.
- **Eventually consistent** - System reaches a consistent data state after some time.

# How to start with MongoDB for free?

#### MongoDB Atlas - The fully managed service for MongoDB deployments in the cloud
For learning and exploring MongoDB in a cloud environment. Basic configuration options:
- 512MB of storage
- Shared RAM
- Upgrade to dedicated clusters for full functionality
- No credit card required to start

#### MongoDB Community -  The source-available, free-to-use, and self-managed version of MongoDB

site: https://www.mongodb.com/

# Connection

In [ ]:
!pip install pymongo

In [1]:
from pymongo import MongoClient

In [2]:
MONGODB_URI = ""

In [3]:
def get_db():
    client = MongoClient(MONGODB_URI)
    return client.workshop_db

In [4]:
db = get_db() 

# CRUD pre-requirements 

In [5]:
from bson.objectid import ObjectId
from datetime import datetime
import pandas as pd

C:\Users\brzyz\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\brzyz\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\brzyz\AppData\Local\Temp/ipykernel_21120/510612029.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Logical operators 

MongoDB | PostgreSQL 
:---: | :---:
`$and` | AND
`$or` | OR
`$not` | NOT

#### Comparison operators 

MongoDB | PostgreSQL 
:---: | :---:
`$eq` | =
`$ne` | <>
`$gt` | >
`$gte` | >=
`$lt` | <
`$lte` | <=
`$in` | IN
`$nin` | NOT IN

# CREATE

#### Create table/collection 

PostgreSQL: `CREATE TABLE students (column1 datatype, column2 datatype, column3 datatype, ....);`
<br>
<br>
MongoDB: `.create-collection("students")` 

In [6]:
db.create_collection("students")

Collection(Database(MongoClient(host=['ac-nh1busv-shard-00-01.gmfvfa0.mongodb.net:27017', 'ac-nh1busv-shard-00-00.gmfvfa0.mongodb.net:27017', 'ac-nh1busv-shard-00-02.gmfvfa0.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-jxx4k6-shard-0', tls=True), 'workshop_db'), 'students')

In [ ]:
schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["_id", "name", "age", "grade"],
        "properties": {
            "name": {
                "bsonType": "string",
                "description": "must be a string and is required"
                },
            "age": {
                "bsonType": "int",
                "minimum": 0,
                "description": "must be a int greater than 0 and is required"
            }, 
            "grade": {
                "bsonType": "double",
                "minimum": 0,
                "maximum": 5,
                "description": "must be a double in [0, 5] and is required"
            }, 
            "classes": { 
                "bsonType": "array",
                "minItems": 1,
                "items": {
                        "bsonType": "string",
                    },
                "description": "must be a array of strings and is not required"
                }
        },
        "additionalProperties": True
    }
}

In [8]:
db.create_collection("students_schema", validator=schema)

Collection(Database(MongoClient(host=['ac-nh1busv-shard-00-01.gmfvfa0.mongodb.net:27017', 'ac-nh1busv-shard-00-00.gmfvfa0.mongodb.net:27017', 'ac-nh1busv-shard-00-02.gmfvfa0.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-jxx4k6-shard-0', tls=True), 'workshop_db'), 'students_schema')

#### Insert data to table/collection

PostgreSQL: `INSERT INTO students (column1, column2, column3, ...) VALUES (value1, value2, value3, ...);`
<br>
<br>
MongoDB: `students.insert_one({record_data})`

In [9]:
db.students.insert_one({"name": "Adam",
                        "age": 24,
                        "grade": 3.8,
                        "active": True,
                        "creation_date": datetime(2022, 1, 10),
                        "graduation_date": None,
                        "classes": ["mathematic", "geography"],
                        "address": {"street": "Marszalkowska 1",
                                    "city": "Warsaw",
                                    "country_code": "PL"},
                        "payment": 1952.43
                        })

InsertOneResult(ObjectId('65d481e726cd4e860ddbb0ee'), acknowledged=True)

PostgreSQL: `INSERT INTO table_name (column1, column2, ...) VALUES (value1, value2, ...), (value1, value2, ...), ...;`
<br>
<br>
MongoDB: `students.insert_many([{}, {}, {}])`

In [10]:
students = [{"name": "Kasia",
             "age": 22,
             "grade": 3.9,
             "active": True,
             "creation_date": datetime(2021, 10, 1),
             "graduation_date": None,
             "classes": ["geography"],
             "address": {"street": "Piaskowa 1",
                         "city": "Warsaw",
                         "country_code": "PL"},
             "payment": 1321.25
            },
            {"name": "Marcin",
             "age": 25,
             "grade": 4.5,
             "active": False,
             "creation_date": datetime(2020, 1, 10),
             "graduation_date": datetime(2024, 1, 2),
             "classes": ["biology", "science"],
             "address": {"street": "Konstruktorska 1",
                         "city": "Warsaw",
                         "country_code": "PL"},
             "payment": 2100.20
            },
            {"name": "Anna",
             "age": 20,
             "grade": 4.2,
             "active": True,
             "creation_date": datetime(2011, 12, 10),
             "graduation_date": None,
             "classes": ["mathematic", "Science"],
             "address": {"street": "Polna 1",
                         "city": "Warsaw",
                         "country_code": "PL"},
             "payment": 980.43
            },
]

db.students.insert_many(students)

InsertManyResult([ObjectId('65d4823426cd4e860ddbb0ef'), ObjectId('65d4823426cd4e860ddbb0f0'), ObjectId('65d4823426cd4e860ddbb0f1')], acknowledged=True)

# READ

PostgreSQL: `SELECT * FROM students;`
<br>
<br>
MongoDB: `students.find()`

In [11]:
students = db.students.find()
students

In [12]:
pd.DataFrame(students)

,_id,name,age,grade,active,creation_date,graduation_date,classes,address,payment
0,65d481e726cd4e860ddbb0ee,Adam,24,3.8,True,2022-01-10,NaT,"[mathematic, geography]","{'street': 'Marszalkowska 1', 'city': 'Warsaw'...",1952.43
1,65d4823426cd4e860ddbb0ef,Kasia,22,3.9,True,2021-10-01,NaT,[geography],"{'street': 'Piaskowa 1', 'city': 'Warsaw', 'co...",1321.25
2,65d4823426cd4e860ddbb0f0,Marcin,25,4.5,False,2020-01-10,2024-01-02,"[biology, science]","{'street': 'Konstruktorska 1', 'city': 'Warsaw...",2100.20
3,65d4823426cd4e860ddbb0f1,Anna,20,4.2,True,2011-12-10,NaT,"[mathematic, Science]","{'street': 'Polna 1', 'city': 'Warsaw', 'count...",980.43


PostgreSQL: `SELECT * students LIMIT 1;`
<br>
<br>
MongoDB: `students.find_one()`

In [15]:
student = db.students.find_one()
student

{'_id': ObjectId('65d481e726cd4e860ddbb0ee'),
 'name': 'Adam',
 'age': 24,
 'grade': 3.8,
 'active': True,
 'creation_date': datetime.datetime(2022, 1, 10, 0, 0),
 'graduation_date': None,
 'classes': ['mathematic', 'geography'],
 'address': {'street': 'Marszalkowska 1',
  'city': 'Warsaw',
  'country_code': 'PL'},
 'payment': 1952.43}

PostgreSQL: `SELECT * FROM students ORDER BY name;`
<br>
<br>
MongoDB: `students.find().sort({"name": 1})`

In [16]:
students = db.students.find().sort({"name": 1})
pd.DataFrame(students)

,_id,name,age,grade,active,creation_date,graduation_date,classes,address,payment
0,65d481e726cd4e860ddbb0ee,Adam,24,3.8,True,2022-01-10,NaT,"[mathematic, geography]","{'street': 'Marszalkowska 1', 'city': 'Warsaw'...",1952.43
1,65d4823426cd4e860ddbb0f1,Anna,20,4.2,True,2011-12-10,NaT,"[mathematic, Science]","{'street': 'Polna 1', 'city': 'Warsaw', 'count...",980.43
2,65d4823426cd4e860ddbb0ef,Kasia,22,3.9,True,2021-10-01,NaT,[geography],"{'street': 'Piaskowa 1', 'city': 'Warsaw', 'co...",1321.25
3,65d4823426cd4e860ddbb0f0,Marcin,25,4.5,False,2020-01-10,2024-01-02,"[biology, science]","{'street': 'Konstruktorska 1', 'city': 'Warsaw...",2100.20


PostgreSQL: `SELECT * FROM students ORDER BY grade DESC LIMIT 2;`
<br>
<br>
MongoDB: `students.find().sort({"grade": -1}).limit(2)`

In [17]:
students = db.students.find().sort({"grade": -1}).limit(2)
pd.DataFrame(students)

,_id,name,age,grade,active,creation_date,graduation_date,classes,address,payment
0,65d4823426cd4e860ddbb0f0,Marcin,25,4.5,False,2020-01-10,2024-01-02,"[biology, science]","{'street': 'Konstruktorska 1', 'city': 'Warsaw...",2100.20
1,65d4823426cd4e860ddbb0f1,Anna,20,4.2,True,2011-12-10,NaT,"[mathematic, Science]","{'street': 'Polna 1', 'city': 'Warsaw', 'count...",980.43


PostgreSQL: `SELECT * FROM students WHERE active = TRUE;`
<br>
<br>
MongoDB: `students.find({"active": True})`

In [18]:
students = db.students.find({"active": True})
pd.DataFrame(students)

,_id,name,age,grade,active,creation_date,graduation_date,classes,address,payment
0,65d481e726cd4e860ddbb0ee,Adam,24,3.8,True,2022-01-10,None,"[mathematic, geography]","{'street': 'Marszalkowska 1', 'city': 'Warsaw'...",1952.43
1,65d4823426cd4e860ddbb0ef,Kasia,22,3.9,True,2021-10-01,None,[geography],"{'street': 'Piaskowa 1', 'city': 'Warsaw', 'co...",1321.25
2,65d4823426cd4e860ddbb0f1,Anna,20,4.2,True,2011-12-10,None,"[mathematic, Science]","{'street': 'Polna 1', 'city': 'Warsaw', 'count...",980.43


PostgreSQL: `SELECT * FROM students WHERE active = TRUE and grade > 4;`
<br>
<br>
MongoDB: `students.find({"active": True, "grade": {"$gt": 4}})`

In [19]:
students = db.students.find({"active": True, "grade": {"$gt": 4}})
pd.DataFrame(students)

,_id,name,age,grade,active,creation_date,graduation_date,classes,address,payment
0,65d4823426cd4e860ddbb0f1,Anna,20,4.2,True,2011-12-10,None,"[mathematic, Science]","{'street': 'Polna 1', 'city': 'Warsaw', 'count...",980.43


PostgreSQL: `SELECT name, grade FROM students WHERE active = TRUE;`
<br>
<br>
MongoDB: `students.find({"active": True}, {"_id": 0 ,"name": 1, "grade": True})`

In [20]:
students = db.students.find({"active": True}, {"_id": 0 ,"name": 1, "grade": True})
pd.DataFrame(students)

,name,grade
0,Adam,3.8
1,Kasia,3.9
2,Anna,4.2


PostgreSQL: `SELECT name, age FROM students WHERE name like 'A%';`
<br>
<br>
MongoDB: `students.find({"name": {"$regex":"^A"}}, {"_id": 0 ,"name": 1, "age": 1})`

In [21]:
students = db.students.find({"name": {"$regex":"^A"}}, {"_id": 0 ,"name": 1, "age": 1})
pd.DataFrame(students)

,name,age
0,Adam,24
1,Anna,20


PostgreSQL: `SELECT active, COUNT(*) FROM students GROUP BY active;`
<br>
<br>
MongoDB: `students.aggregate([{"$group": {"_id": "$active", "number_of_students": {"$sum": 1}}}])`

In [22]:
students=db.students.aggregate([{"$group": {"_id": "$active", "number_of_students": {"$sum": 1}}}])
pd.DataFrame(students)

,_id,number_of_students
0,False,1
1,True,3


PostgreSQL: `SELECT active, SUM(payment) FROM students GROUP BY active;`
<br>
<br>
MongoDB: `students.aggregate([{"$group": {"_id": "$active",  "total_fee": {"$sum": "$payment"}}}])`

In [23]:
students=db.students.aggregate([{"$group": {"_id": "$active", "total_fee": {"$sum": "$payment"}}}])
pd.DataFrame(students)

,_id,total_fee
0,True,4254.11
1,False,2100.20


In [24]:
rate_to_USD = 0.25
students=db.students.aggregate([{"$group": {
    "_id": "$active",
    "total_fee_in_USD": {"$sum": {"$multiply": ["$payment", rate_to_USD]}}
}}])
pd.DataFrame(students)

,_id,total_fee_in_USD
0,True,1063.5275
1,False,525.0500


# UPDATE

PostgreSQL: `UPDATE students SET active = TRUE WHERE name = 'Marcin';`
<br>
<br>
MongoDB: `students.update_one({"name": "Marcin"}, {"$set": {"active": True}})`

In [25]:
db.students.update_one({"name": "Marcin"}, {"$set": {"active": True}})

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000288'), 'opTime': {'ts': Timestamp(1708426517, 1000), 't': 648}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708426517, 1000), 'signature': {'hash': b'\xd3<\x92\x86\xad\xd0\x9e"&\xb2\xc5\x91\x05\x89\xf3\xd0\x9d\xbc\xbb\xd9', 'keyId': 7302022492913139713}}, 'operationTime': Timestamp(1708426517, 1000), 'updatedExisting': True}, acknowledged=True)

PostgreSQL: `UPDATE students SET active = False WHERE _id = id_number;`
<br>
<br>
MongoDB: `students.update_one({"_id": ObjectId(id_number)}, {"$set": {"active": False}})`

In [26]:
db.students.update_one({"_id": ObjectId("65d481e726cd4e860ddbb0ee")}, {"$set": {"active": False}})

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000288'), 'opTime': {'ts': Timestamp(1708426557, 45), 't': 648}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708426557, 45), 'signature': {'hash': b'\xcf9\xab\xbfDLy(Ux\xe4\xc5\xc2\x1b~\xb2<3\xa2\xdc', 'keyId': 7302022492913139713}}, 'operationTime': Timestamp(1708426557, 45), 'updatedExisting': True}, acknowledged=True)

PostgreSQL: `UPDATE students SET active = NULL WHERE _id = id_number;`
<br>
<br>
MongoDB: `students.update_one({"_id": ObjectId(id_number)}, {"$unset": {"active": ""}})`

In [27]:
db.students.update_one({"_id": ObjectId("65d481e726cd4e860ddbb0ee")}, {"$unset": {"active": ""}})

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000288'), 'opTime': {'ts': Timestamp(1708426613, 32), 't': 648}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708426613, 32), 'signature': {'hash': b'\xae\xd8o\x18C?\x97\x85y\xe9f\xff\xc6\xca\xd3\xd9\xa9\x98\x11#', 'keyId': 7302022492913139713}}, 'operationTime': Timestamp(1708426613, 32), 'updatedExisting': True}, acknowledged=True)

PostgreSQL: `UPDATE students SET active = NULL;`
<br>
<br>
MongoDB: `students.update_many({}, {"$unset": {"active": ""}})`

In [28]:
db.students.update_many({}, {"$unset": {"active": ""}})

UpdateResult({'n': 4, 'electionId': ObjectId('7fffffff0000000000000288'), 'opTime': {'ts': Timestamp(1708426789, 28), 't': 648}, 'nModified': 3, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708426789, 28), 'signature': {'hash': b'Tv&\xc1i`\x0bo\xb5/&|".\x92\xa0\xaf\x88\xf4)', 'keyId': 7302022492913139713}}, 'operationTime': Timestamp(1708426789, 28), 'updatedExisting': True}, acknowledged=True)

PostgreSQL: `UPDATE students SET active = TRUE WHERE creation_date >= '2022-01-01';`
<br>
<br>
MongoDB: `students.update_many({"creation_date": {"$gte": datetime(2022, 1, 1)}}, {"$set": {"active": True}})`

In [29]:
db.students.update_many({"creation_date": {"$gte": datetime(2022, 1, 1)}}, {"$set": {"active": True}})

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000288'), 'opTime': {'ts': Timestamp(1708426829, 11), 't': 648}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708426829, 11), 'signature': {'hash': b'<\x15\xea\xc1\xd4\xae\x8b\xdfB\xe4\x98\xca\xd8\n,\x08\x03u\x84\xc2', 'keyId': 7302022492913139713}}, 'operationTime': Timestamp(1708426829, 11), 'updatedExisting': True}, acknowledged=True)

PostgreSQL: `UPDATE students SET active = FALSE WHERE active is NULL;`
<br>
<br>
MongoDB: `students.update_many({"active": {"$exists": False}}, {"$set": {"active": False}})`

In [30]:
db.students.update_many({"active": {"$exists": False}}, {"$set": {"active": False}})

UpdateResult({'n': 3, 'electionId': ObjectId('7fffffff0000000000000288'), 'opTime': {'ts': Timestamp(1708426874, 42), 't': 648}, 'nModified': 3, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708426874, 42), 'signature': {'hash': b'\xd1\x82\xf1cT\x18>\\\x8bZ%\x88\xf8}\x8d\xe7\xa1\x0f1\xd5', 'keyId': 7302022492913139713}}, 'operationTime': Timestamp(1708426874, 42), 'updatedExisting': True}, acknowledged=True)

# DELETE

PostgreSQL: `DELETE FROM students WHERE name = 'Adam';`
<br>
<br>
MongoDB: `students.delete_one({"name": "Adam"})`

In [ ]:
db.students.delete_one({"name": "Adam"})

PostgreSQL: `DELETE FROM students WHERE active = FALSE;`
<br>
<br>
MongoDB: `students.delete_many({"active": False})`

In [ ]:
db.students.delete_many({"active": False})

PostgreSQL: ` DROP TABLE students_schema;`
<br>
<br>
MongoDB: `students_schema.drop()`

In [ ]:
db.students_schema.drop()

# INDEXES

PostgreSQL: `EXPLAIN SELECT * FROM students WHERE name = 'Adam' and grade > 4;`
<br>
<br>
MongoDB: `students.find({"name": "Adam", "grade": {"$gt": 4}}).explain()`

In [31]:
db.students.find({"name": "Adam", "grade": {"$gt": 4}}).explain()

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'workshop_db.students',
  'indexFilterSet': False,
  'parsedQuery': {'$and': [{'name': {'$eq': 'Adam'}}, {'grade': {'$gt': 4}}]},
  'queryHash': 'CFDE3782',
  'planCacheKey': 'CFDE3782',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'$and': [{'name': {'$eq': 'Adam'}}, {'grade': {'$gt': 4}}]},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 0,
  'executionTimeMillis': 0,
  'totalKeysExamined': 0,
  'totalDocsExamined': 4,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'$and': [{'name': {'$eq': 'Adam'}}, {'grade': {'$gt': 4}}]},
   'nReturned': 0,
   'executionTimeMillisEstimate': 0,
   'works': 5,
   'advanced': 0,
   'needTime': 4,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'direction': 'forward',
 

PostgreSQL: `CREATE INDEX name_1 ON students (name);`
<br>
<br>
MongoDB: `students.create_index({"name": 1})`

In [32]:
db.students.create_index({"name": 1})

'name_1'

PostgreSQL: `EXPLAIN SELECT * FROM students WHERE name = 'Adam' and grade > 4;`
<br>
<br>
MongoDB: `students.find({"name": "Adam", "grade": {"$gt": 4}}).explain()`

In [33]:
db.students.find({"name": "Adam", "grade": {"$gt": 4}}).explain()

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'workshop_db.students',
  'indexFilterSet': False,
  'parsedQuery': {'$and': [{'name': {'$eq': 'Adam'}}, {'grade': {'$gt': 4}}]},
  'queryHash': 'CFDE3782',
  'planCacheKey': '89214452',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'FETCH',
   'filter': {'grade': {'$gt': 4}},
   'inputStage': {'stage': 'IXSCAN',
    'keyPattern': {'name': 1},
    'indexName': 'name_1',
    'isMultiKey': False,
    'multiKeyPaths': {'name': []},
    'isUnique': False,
    'isSparse': False,
    'isPartial': False,
    'indexVersion': 2,
    'direction': 'forward',
    'indexBounds': {'name': ['["Adam", "Adam"]']}}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 0,
  'executionTimeMillis': 1,
  'totalKeysExamined': 1,
  'totalDocsExamined': 1,
  'executionStages': {'stage': 'FETCH',
   'filter': {'grade': {'$gt'

PostgreSQL: `SELECT * FROM pg_indexes WHERE tablename = 'students';`
<br>
<br>
MongoDB: `students.index_information()`

In [34]:
db.students.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_1': {'v': 2, 'key': [('name', 1)]}}

# Relations

# Transactions